In [22]:
import random
from faker import Faker
import pymysql  # For MySQL
import pyodbc   # For SQL Server
import time

# Initialize Faker
fake = Faker()


In [23]:
# Database connection variables
DB_TYPE = 'mysql'  # Change to 'sqlserver' for SQL Server
HOST = 'localhost'
USER = 'root'
PASSWORD = 'root'
DATABASE = 'facebook'

In [24]:
# Data variables
NUM_USERS = 10000
NUM_POSTS = 100000
NUM_LIKES = 1000000
NUM_COMMENTS = 500000

In [25]:
# Progress intervals for each table (5%)
PROGRESS_INTERVAL_USERS = NUM_USERS // 20   # 5% of 1000 is 50
PROGRESS_INTERVAL_POSTS = NUM_POSTS // 20   # 5% of 10000 is 500
PROGRESS_INTERVAL_LIKES = NUM_LIKES // 20   # 5% of 100000 is 5000
PROGRESS_INTERVAL_COMMENTS = NUM_COMMENTS // 20  # 5% of 5000 is 250


In [26]:
# Connect to the database
def connect_to_db():
    if DB_TYPE == 'mysql':
        conn = pymysql.connect(host=HOST, user=USER, password=PASSWORD, database=DATABASE)
    elif DB_TYPE == 'sqlserver':
        conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                              f'SERVER={HOST};DATABASE={DATABASE};UID={USER};PWD={PASSWORD}')
    return conn
# Insert users with progress tracking
def insert_users(conn, num_users):
    cursor = conn.cursor()
    print(f"Inserting {num_users} users...")

    for i in range(1, num_users + 1):
        name = fake.first_name()  # Single word name
        email = fake.email()[:50]  # Ensure email is small
        password = fake.password(length=12)  # Exactly 12 characters for password
        phone = fake.phone_number()[:15]  # Ensure phone is within limits

        cursor.execute("INSERT INTO users (name, email, password, phone) VALUES (%s, %s, %s, %s)",
                       (name, email, password, phone))

        if i % PROGRESS_INTERVAL_USERS == 0:
            print(f"{i}/{num_users} users inserted ({(i / num_users) * 100:.2f}% complete)")
    
    conn.commit()

# Insert posts with progress tracking
def insert_posts(conn, num_posts, num_users):
    cursor = conn.cursor()
    print(f"Inserting {num_posts} posts...")

    for i in range(1, num_posts + 1):
        user_id = random.randint(1, num_users)
        content = fake.sentence(nb_words=10)[:255]  # Limited content to 255 characters

        cursor.execute("INSERT INTO posts (user_id, content) VALUES (%s, %s)", (user_id, content))

        if i % PROGRESS_INTERVAL_POSTS == 0:
            print(f"{i}/{num_posts} posts inserted ({(i / num_posts) * 100:.2f}% complete)")
    
    conn.commit()

# Insert likes with progress tracking
def insert_likes(conn, num_likes, num_users, num_posts):
    cursor = conn.cursor()
    print(f"Inserting {num_likes} likes...")

    for i in range(1, num_likes + 1):
        user_id = random.randint(1, num_users)
        post_id = random.randint(1, num_posts)

        cursor.execute("INSERT INTO likes (post_id, user_id) VALUES (%s, %s)", (post_id, user_id))

        if i % PROGRESS_INTERVAL_LIKES == 0:
            print(f"{i}/{num_likes} likes inserted ({(i / num_likes) * 100:.2f}% complete)")
    
    conn.commit()

# Insert comments with progress tracking
def insert_comments(conn, num_comments, num_users, num_posts):
    cursor = conn.cursor()
    print(f"Inserting {num_comments} comments...")

    for i in range(1, num_comments + 1):
        user_id = random.randint(1, num_users)
        post_id = random.randint(1, num_posts)
        comment = fake.sentence(nb_words=15)[:255]  # Limited comment to 255 characters

        cursor.execute("INSERT INTO comments (post_id, user_id, comment) VALUES (%s, %s, %s)", 
                       (post_id, user_id, comment))

        if i % PROGRESS_INTERVAL_COMMENTS == 0:
            print(f"{i}/{num_comments} comments inserted ({(i / num_comments) * 100:.2f}% complete)")
    
    conn.commit()

# Function to delete all data from the database
def delete_all_data(conn):
    cursor = conn.cursor()
    print("Deleting all data from the database...")

    if DB_TYPE == 'mysql':
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")  # Disable foreign key checks for MySQL

    # List of tables in the order they should be truncated
    tables = ['comments', 'likes', 'posts', 'users']

    for table in tables:
        cursor.execute(f"TRUNCATE TABLE {table};")  # Truncate table to delete all rows
        print(f"Table {table} truncated.")

    if DB_TYPE == 'mysql':
        cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")  # Re-enable foreign key checks for MySQL

    conn.commit()
    print("All data has been deleted.")




In [27]:
# Main function
def main():
    conn = connect_to_db()
    start_time = time.time()
    
    try:
        delete_all_data(conn)
        insert_users(conn, NUM_USERS)
        insert_posts(conn, NUM_POSTS, NUM_USERS)
        insert_likes(conn, NUM_LIKES, NUM_USERS, NUM_POSTS)
        insert_comments(conn, NUM_COMMENTS, NUM_USERS, NUM_POSTS)
    finally:
        conn.close()
        end_time = time.time()
        print(f"Data insertion completed in {round(end_time - start_time, 2)} seconds.")


In [28]:
if __name__ == '__main__':
    main()

Deleting all data from the database...
Table comments truncated.
Table likes truncated.
Table posts truncated.
Table users truncated.
All data has been deleted.
Inserting 10000 users...
500/10000 users inserted (5.00% complete)
1000/10000 users inserted (10.00% complete)
1500/10000 users inserted (15.00% complete)
2000/10000 users inserted (20.00% complete)
2500/10000 users inserted (25.00% complete)
3000/10000 users inserted (30.00% complete)
3500/10000 users inserted (35.00% complete)
4000/10000 users inserted (40.00% complete)
4500/10000 users inserted (45.00% complete)
5000/10000 users inserted (50.00% complete)
5500/10000 users inserted (55.00% complete)
6000/10000 users inserted (60.00% complete)
6500/10000 users inserted (65.00% complete)
7000/10000 users inserted (70.00% complete)
7500/10000 users inserted (75.00% complete)
8000/10000 users inserted (80.00% complete)
8500/10000 users inserted (85.00% complete)
9000/10000 users inserted (90.00% complete)
9500/10000 users inserte